<a href="https://colab.research.google.com/github/Meenusj/Case_study/blob/main/testlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=132aa68bff7d22252fd8accf5ad1d1d3f927de17a5495049920a8d3d8a52c1dc
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Replace 'FILE_ID' with the ID of your file
fasttext.bin = 'https://drive.google.com/file/d/1uUz9v-enc3qfeEiDBVVVN3gPK7lRqHbG/view?usp=sharing'
file_path = '/content/drive/My Drive/{}'.format(file_id)

# Now you can use the file_path to access your file in Colab



In [ ]:
file_path = 'https://drive.google.com/file/d/1RSACqkEp5tsaJT0A_Sgjdrx0YXytfy_2/view?usp=sharing'


In [ ]:
!gdown --id 1RSACqkEp5tsaJT0A_Sgjdrx0YXytfy_2


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1RSACqkEp5tsaJT0A_Sgjdrx0YXytfy_2

but Gdown can't. Please check connections and permissions.


In [ ]:
import shutil

destination_path = '/content/fasttext.bin'
shutil.copy(file_path, destination_path)


FileNotFoundError: [Errno 2] No such file or directory: 'https://drive.google.com/file/d/1RSACqkEp5tsaJT0A_Sgjdrx0YXytfy_2/view?usp=sharing'

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
import fasttext
import nltk
import pickle

nltk.download('punkt')

# Load the FastText model
model = fasttext.load_model('fasttext.bin')

# Assuming the correct column name is 'text', replace 'tweet_text' with the actual name
def preprocess(text):
    # Tokenization
    tokens = text.split()

    # Case conversion
    tokens = [word.lower() for word in tokens]

    # Remove hashtags and usernames
    tokens = [re.sub(r'#\w+|@\w+', '', word) for word in tokens]

    # Remove punctuation
    tokens = [word for word in tokens if word.isalnum()]

    return ' '.join(tokens)

# Load the pre-trained LSTM model
loaded_model = load_model('/content/lstm_model (1).h5')

# Load the label encoder
with open('/content/label_encoder (3).pkl', 'rb') as le_file:
    label_encoder = pickle.load(le_file)

# Load your new dataset (replace 'new_test.csv' with your actual dataset file)
new_csv_file_path = 'test.csv'
new_df = pd.read_csv(new_csv_file_path, delimiter=';')

# Preprocess the text in the 'text' column
new_df['preprocessed_text'] = new_df['text'].apply(preprocess)

# Tokenize the text
new_tokenized_text = [word_tokenize(text) for text in new_df['preprocessed_text']]


# Get word vectors for each token using the pre-trained FastText model
new_word_vectors = [model.get_word_vector(word) for tokens in new_tokenized_text for word in tokens]


# Convert word vectors to DataFrame
new_word_vectors_df = pd.DataFrame(new_word_vectors, columns=[f'feature_{i}' for i in range(300)])

# Concatenate the original DataFrame with the word vectors DataFrame
new_df_with_vectors = pd.concat([new_df, new_word_vectors_df], axis=1)

# Extract feature columns (assuming they start from column 'feature_0')
new_feature_columns = new_df_with_vectors.columns[new_df_with_vectors.columns.str.startswith('feature_')]
# Extract features and labels
new_X = new_df_with_vectors[new_feature_columns].values
new_y_actual = label_encoder.transform(new_df_with_vectors['class_type'])

# Reshape the input data to be compatible with Conv1D layer
new_X = new_X.reshape(new_X.shape[0], new_X.shape[1], 1)
# Predict on the new dataset
new_y_pred_probs = loaded_model.predict(new_X)
new_y_pred_classes = new_y_pred_probs.argmax(axis=-1)

# Calculate evaluation metrics for the new dataset
new_accuracy = accuracy_score(new_y_actual, new_y_pred_classes)
new_precision = precision_score(new_y_actual, new_y_pred_classes, average='weighted')
new_recall = recall_score(new_y_actual, new_y_pred_classes, average='weighted')
new_f1 = f1_score(new_y_actual, new_y_pred_classes, average='weighted')

# Print the evaluation metrics for the new dataset
print(f"New Dataset Evaluation Metrics:")
print(f"Accuracy: {new_accuracy:.4f}")
print(f"Precision: {new_precision:.4f}")
print(f"Recall: {new_recall:.4f}")
print(f"F1 Score: {new_f1:.4f}")

# Create a confusion matrix for the new dataset
new_conf_matrix = confusion_matrix(new_y_actual, new_y_pred_classes)

# Print the confusion matrix for the new dataset
print("Confusion Matrix for the New Dataset:")
print(new_conf_matrix)

# Create a classification report for the new dataset
new_class_report = classification_report(new_y_actual, new_y_pred_classes)

# Print the classification report for the new dataset
print("Classification Report for the New Dataset:")
print(new_class_report)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


ValueError: fasttext.bin has wrong file format!